In [66]:
from LinkedBinaryTree import * 
import random
from game_tree import *

class MoriZwanzigMemory:
    """Memory module for tracking and utilizing past states."""
    
    def __init__(self, max_length=10):
        """Initialize the memory with a maximum length."""
        self.max_length = max_length
        self.history = []

    def update(self, node):
        """Add the current node to memory history."""
        self.history.append(node)
        if len(self.history) > self.max_length:
            self.history.pop(0)  # Remove the oldest node to maintain max_length

    def compute_effect(self):
        """Compute memory effect based on history."""
        if not self.history:
            return 0.0  # No effect if no history exists
        
        # Use an exponentially decaying weight for historical states
        weights = [1 / (i + 1) for i in range(len(self.history))]
        weighted_effect = sum(node.right_weight * weights[i] for i, node in enumerate(self.history))
        # Penalize if the player has consistently low performance
        performance_trend = sum(node.right_weight for node in self.history) / len(self.history)
        if performance_trend < 0.5:  # Below-average performance
            return average_effect * -1  # Penalize the effect

        return weighted_effect / sum(weights)  # Normalize the effect

def build_set_tree_with_weights(tree, mem_len, match_data):
    construct_tree(tree)
    visited = set()
    memory = MoriZwanzigMemory(max_length= mem_len)
    
    def assign_weights(node, weight_scaler, momentum):
        if node is None or node in visited:
            return

        visited.add(node)
        memory.update(node)
        
        # Determine who has momentum and advantage
        server = int(match_data.get("server", 0))
        point_victor = int(match_data.get("point_victor", 0))
        p1_break_pt = int(match_data.get("p1_break_pt_won", 0))
        p2_break_pt = int(match_data.get("p2_break_pt_won", 0))

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            if server==1:
                node.right_weight = 0.52
                node.left_weight = 0.48
            else: 
                node.right_weight = 0.48
                node.left_weight =0.52

            if server == 2 and p1_break_pt==1:
                node.right_weight += momentum 
            elif server == 1 and p2_break_pt==1:
                node.left_weight += momentum

            # Apply memory effect to right_weight
            memory_effect = memory.compute_effect()
            node.right_weight += weight_scaler*memory_effect
            
            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight

        # Debugging output for specific scores
        if node.data in ["15-15", "15-30"]:
            print(f"Node: {node.data}, Right Weight: {node.right_weight}, Left Weight: {node.left_weight}")

        assign_weights(node.left, weight_scaler, momentum)
        assign_weights(node.right, weight_scaler, momentum)

    assign_weights(tree.root, 0.5, 0.05)

def find_start_node(tree, current_score):
    if tree.root is None:
        raise ValueError("The tree root is None. Ensure the tree is constructed properly.")

    stack = [tree.root]
    visited = set()
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        if node.data == current_score:
            return node
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    raise ValueError(f"Score {current_score} not found in tree. Verify tree construction and current_score format.")

def predict_set_winner_simulation(tree, match_data):
    player_games = int(match_data["p1_games"])
    opponent_games = int(match_data["p2_games"])
    player_score = match_data["p1_score"]
    opponent_score = match_data["p2_score"]
    current_score = f"{player_score}-{opponent_score}"

    current_node = find_start_node(tree, current_score)
    if current_node is None:
        raise ValueError(f"Score {current_score} not found in tree")

    # print(f"Starting simulation at score {current_score}")

    while True:
        # print(f"Current Node: {current_node.data}, Player Games: {player_games}, Opponent Games: {opponent_games}")

        # Check for set victory
        if player_games >= 6 and player_games - opponent_games >= 2:
            # print("Player 1 wins the set!")
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            # print("Player 2 wins the set!")
            return "Opponent wins the set", player_games, opponent_games

        # Simulate point outcome
        random_value = random.random()
        
        if current_node.data == "WIN":
            # print("Player 1 wins a game!")
            player_games += 1
            current_node = tree.root  # Restart at the root for a new game
            continue
        elif current_node.data == "LOSE":
            # print("Player 2 wins a game!")
            opponent_games += 1
            current_node = tree.root  # Restart at the root for a new game
            continue

        # Transition based on weights
        if random_value < current_node.right_weight and current_node.right:
            current_node = current_node.right
            # print(f"Transitioned to: {current_node.data} (Right Child)")
        elif current_node.left:
            current_node = current_node.left
            # print(f"Transitioned to: {current_node.data} (Left Child)")
        else:
            print("No valid child node. Ending simulation.")
            break


In [67]:

import pandas as pd
from collections import Counter

# Load match data
match_data_df = pd.read_csv("m1.csv")

# Convert the first row of the DataFrame to a dictionary
match_data = match_data_df.iloc[-1].to_dict()

print(match_data)
# Initialize the binary tree
binary_tree = LinkedBinaryTree()

# Build the tree with weights using the match data
build_set_tree_with_weights(binary_tree, 5, match_data)


results = []
for _ in range(100):
    result, player_games, opponent_games = predict_set_winner_simulation(binary_tree, match_data)
    results.append((result, player_games, opponent_games))

# Count the number of times Player 1 wins
player1_wins = sum(1 for result, _, _ in results if result == "Player wins the set")

# Get distribution of scores
score_distribution = Counter((player_games, opponent_games) for _, player_games, opponent_games in results)

print(f"Player 1 Wins: {player1_wins}")
print("Score Distribution:")
for score, count in score_distribution.items():
    print(f"{score}: {count}")

{'match_id': '2023-wimbledon-1301', 'player1': 'Carlos Alcaraz', 'player2': 'Nicolas Jarry', 'elapsed_time': '00:33:58', 'set_no': 1, 'game_no': 8, 'point_no': 49, 'p1_sets': 0, 'p2_sets': 0, 'p1_games': 4, 'p2_games': 3, 'p1_score': '15', 'p2_score': '30', 'server': 2, 'serve_no': 2, 'point_victor': 1, 'p1_points_won': 25, 'p2_points_won': 24, 'game_victor': 0, 'set_victor': 0, 'p1_ace': 0, 'p2_ace': 0, 'p1_winner': 0, 'p2_winner': 0, 'winner_shot_type': '0', 'p1_double_fault': 0, 'p2_double_fault': 0, 'p1_unf_err': 0, 'p2_unf_err': 1, 'p1_net_pt': 0, 'p2_net_pt': 0, 'p1_net_pt_won': 0, 'p2_net_pt_won': 0, 'p1_break_pt': 0, 'p2_break_pt': 0, 'p1_break_pt_won': 0, 'p2_break_pt_won': 0, 'p1_break_pt_missed': 0, 'p2_break_pt_missed': 0, 'p1_distance_run': 17.838, 'p2_distance_run': 18.464, 'rally_count': 6, 'speed_mph': 101.0, 'serve_width': 'BW', 'serve_depth': 'NCTL', 'return_depth': 'D'}
Node: 15-30, Right Weight: 0.6021010977401099, Left Weight: 0.39789890225989
Node: 15-15, Right We